In [23]:

import pandas as pd  
import numpy as np   

In [21]:
import os
import re

try:
    import cPickle as pickle
except:
    import pickle

In [24]:
data = pd.read_csv('data_coding_exercise.txt', sep="\t", header = None, names=["Uastr", "browser", "version"])
data.head() 

,Uastr,browser,version
0,Mozilla/5.0 (Windows NT 6.2) AppleWebKit/537.3...,Chrome,39
1,Mozilla/5.0 (Linux; Android 4.1.1; GT-N8010 Bu...,Chrome,54
2,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,Chrome,39
3,Mozilla/5.0 (Windows NT 6.3; Win64; x64) Apple...,Chrome,45
4,Mozilla/5.0 (Windows NT 6.2) AppleWebKit/537.3...,Chrome,37


In [27]:
browDict = {"Chrome":0, "Chrome Mobile":1, "Chrome Mobile iOS":2,
"Safari":3, "Mobile Safari":4,
"IE":5, "IE Mobile":6,
"Edge":7, "Edge Mobile":8,
"Firefox":9, "Firefox Mobile":10, "Firefox iOS":11,
"Opera":12, "Opera Mobile":13, "Opera Mini":14,
"UC Browser":15, "Sogou Explorer":16,
"QQ Browser":17, "QQ Browser Mobile":18,
"Maxthon":19, "AOL":20, "Facebook":21,
"AppleMail":22, "Puffin":23, "Android":24, "YandexSearch":25, "BlackBerry WebKit":26, "Amazon Silk":27}

check the data.

In [29]:
uas_df = data.Uastr
uas_df.head() 

0    Mozilla/5.0 (Windows NT 6.2) AppleWebKit/537.3...
1    Mozilla/5.0 (Linux; Android 4.1.1; GT-N8010 Bu...
2    Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...
3    Mozilla/5.0 (Windows NT 6.3; Win64; x64) Apple...
4    Mozilla/5.0 (Windows NT 6.2) AppleWebKit/537.3...
Name: Uastr, dtype: object

In [30]:
keyWords = ["Mozilla", "Windows", "AppleWebKit", "KHTML", "Chrome", "Safari", "Linux", "GT-N8010", "WOW64", 
            "Win64", "Mobile", "FBAV", "Trident", "Lenovo", "en-gb", "HUAWEI",  "UCBrowser", "U3", "MSIE", "IEMobile",
            "GSA", "Firefox", "FxiOS", "Tablet", "Gecko", "YandexSearch", "like", "QQBrowser", "Macintosh", "Micromax", "hola_android",
           "Android 2", "Android 4", "Android 5", "Android 6", "MetaSr" "Edge", "Opera Mini","OPR", "MQQBrowser", "BB10", 
            "Intel Mac OS X 10", "AOL", "CriOS", "BlackBerry", "Puffin", "Silk", "SE 2.X MetaSr"]

In [53]:
print(data.shape)
print(len(keyWords))


(421215, 3)
47


Build the attributes matrix for features

In [54]:
strs = ['' for x in range((data.shape)[0])]

In [58]:
for i in range(0, len(uas_df)):
    str1 = uas_df.iloc[i]
    st = ''
    for word in keyWords:
        if str1.find(word) >= 0:
            if len(st) == 0: st = word
            else:
                st = st +'|' + word
    strs[i] = st;


In [59]:
strs[0]

'Mozilla|Windows|AppleWebKit|KHTML|Chrome|Safari|Gecko|like'

Digitize the y

In [64]:
y1 = np.zeros((data.shape)[0])

In [66]:
for i in range(0, len(uas_df)):
    str1 = data.browser.iloc[i]
    y1[i] = browDict[str1]

the feature matrix, string to num

In [75]:
data_1 = pd.DataFrame({'atts':strs})

In [83]:
data_x = data_1.atts.str.get_dummies(sep='|')

In [84]:
data_x.head()

,AOL,Android 2,Android 4,Android 5,Android 6,AppleWebKit,BB10,BlackBerry,Chrome,CriOS,...,Trident,U3,UCBrowser,WOW64,Win64,Windows,YandexSearch,en-gb,hola_android,like
0,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1
1,0,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,1,0,0,1,0,...,0,0,0,1,0,1,0,0,0,1
3,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,1,0,0,0,1
4,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1


In [85]:
data_y = pd.DataFrame({'y':y1})

In [87]:
data_new = pd.concat([data_y, data_x], axis=1) 

In [88]:
#from sklearn.linear_model import SGDClassifier
import sklearn

train, test = sklearn.cross_validation.train_test_split(data_new, train_size = 0.7)


In [96]:
from sklearn.linear_model import SGDClassifier

Xtrain = train.values
Xtest = test.values


In [103]:

train_x = Xtrain[:,1:]
train_y = Xtrain[:,0]
test_x  = Xtest[:,1:]
test_y  = Xtest[:,0]

Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [105]:
from sklearn import linear_model

clf = linear_model.SGDClassifier();
clf.fit(train_x, train_y)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [106]:
y_test_pred = clf.predict(test_x)

In [107]:
ratio = sum(y_test_pred == test_y)/test_y.shape[0]

In [108]:
print(ratio)

0.994547540854


In [109]:
browDictRev = {0:"Chrome", 1:"Chrome Mobile", 2:"Chrome Mobile iOS", 3:"Safari", 4:"Mobile Safari", 5:"IE", 6:"IE Mobile",
        7:"Edge", 8:"Edge Mobile", 9:"Firefox", 10:"Firefox Mobile", 11:"Firefox iOS", 12:"Opera", 13:"Opera Mobile", 14:"Opera Mini",
        15:"UC Browser", 16:"Sogou Explorer", 17:"QQ Browser", 18:"QQ Browser Mobile", 19:"Maxthon", 20:"AOL", 21:"Facebook",
        22:"AppleMail", 23:"Puffin", 24:"Android", 25:"YandexSearch", 26:"BlackBerry WebKit", 27:"Amazon Silk"}

Load the data in test file

In [128]:
test_data = pd.read_csv('test_data_coding_exercise.txt', sep="\t", header = None, names=["Uastr", "browser", "version"])
uas_df_test = test_data.Uastr

row = int((test_data.shape)[0])
col = int(data_x.shape[1])

val = train_x[0:row,:]
val.fill(0)
val.shape


(2, 46)

In [136]:
print(data_x.columns.values)

['AOL' 'Android 2' 'Android 4' 'Android 5' 'Android 6' 'AppleWebKit' 'BB10'
 'BlackBerry' 'Chrome' 'CriOS' 'FBAV' 'Firefox' 'FxiOS' 'GSA' 'GT-N8010'
 'Gecko' 'HUAWEI' 'IEMobile' 'Intel Mac OS X 10' 'KHTML' 'Lenovo' 'Linux'
 'MQQBrowser' 'MSIE' 'Macintosh' 'Micromax' 'Mobile' 'Mozilla' 'OPR'
 'Opera Mini' 'Puffin' 'QQBrowser' 'SE 2.X MetaSr' 'Safari' 'Silk' 'Tablet'
 'Trident' 'U3' 'UCBrowser' 'WOW64' 'Win64' 'Windows' 'YandexSearch'
 'en-gb' 'hola_android' 'like']


Build the test data matrix with the same order as the training matrix

In [138]:
names = data_x.columns.values

for i in range(0, len(uas_df_test)):
    str1 = uas_df_test.iloc[i]
    st = ''
    for word in keyWords:
        if str1.find(word) >= 0:
            for j in range(0, col):
                if names[j] == word:
                    val[i][j] = 1

y1_test = np.zeros((test_data.shape)[0])
for i in range(0, len(uas_df_test)):
    str1 = test_data.browser.iloc[i]
    y1_test[i] = browDict[str1]
    
data_x_test

y_test_pred = clf.predict(val)
print(y_test_pred)

[ 0.  0.]


In [142]:
strs_pred = ['' for x in range(len(y_test_pred))]
strs_ver = ['null' for x in range(len(y_test_pred))]

for i in range(len(y_test_pred)):
    strs_pred[i] = browDictRev[y_test_pred[i]];
    
print(strs_pred)

['Chrome', 'Chrome']


In [143]:
data_test_pred = pd.DataFrame({'pred':strs_pred})
data_new_test = pd.concat([test_data, data_test_pred], axis=1) 
data_new_test.shape

(2, 4)

In [144]:
data_test_ver = pd.DataFrame({'ver_pred':strs_ver})
data_new_test = pd.concat([data_new_test, data_test_ver], axis=1) 
data_new_test.shape

(2, 5)

In [145]:
data_new_test.to_csv("test_result_predict.txt", header=None, index=None, sep='\t')